#順列と組みあわせ
結晶格子の座標をプログラムで作りだす場合を考えてみましょう。2次元の10x10の単純正方格子の座標は、2重ループで簡単に書けます。

In [2]:
pos = []
for x in range(10):
    for y in range(10):
        pos.append((x,y))
print(pos)

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9)]


同じように、4x4x4の3次元の立方格子の座標は3重ループで書けます。

In [3]:
pos = []
for x in range(4):
    for y in range(4):
        for z in range(4):
            pos.append((x,y,z))
print(pos)

[(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 0, 3), (0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 1, 3), (0, 2, 0), (0, 2, 1), (0, 2, 2), (0, 2, 3), (0, 3, 0), (0, 3, 1), (0, 3, 2), (0, 3, 3), (1, 0, 0), (1, 0, 1), (1, 0, 2), (1, 0, 3), (1, 1, 0), (1, 1, 1), (1, 1, 2), (1, 1, 3), (1, 2, 0), (1, 2, 1), (1, 2, 2), (1, 2, 3), (1, 3, 0), (1, 3, 1), (1, 3, 2), (1, 3, 3), (2, 0, 0), (2, 0, 1), (2, 0, 2), (2, 0, 3), (2, 1, 0), (2, 1, 1), (2, 1, 2), (2, 1, 3), (2, 2, 0), (2, 2, 1), (2, 2, 2), (2, 2, 3), (2, 3, 0), (2, 3, 1), (2, 3, 2), (2, 3, 3), (3, 0, 0), (3, 0, 1), (3, 0, 2), (3, 0, 3), (3, 1, 0), (3, 1, 1), (3, 1, 2), (3, 1, 3), (3, 2, 0), (3, 2, 1), (3, 2, 2), (3, 2, 3), (3, 3, 0), (3, 3, 1), (3, 3, 2), (3, 3, 3)]


1〜10のなかから異なる2つを選んだ組みあわせを作りたい場合は、条件分けすればいいのです。

In [4]:
s = []
for i in range(1,11):
    for j in range(1,11):
        if i < j:
            s.append((i,j))
print(s)

[(1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 7), (6, 8), (6, 9), (6, 10), (7, 8), (7, 9), (7, 10), (8, 9), (8, 10), (9, 10)]


でも、jの繰り返し範囲を調節すれば、if文も要らなくなります。

In [5]:
s = []
for i in range(1,11):
    for j in range(i+1,11):
        s.append((i,j))
print(s)

[(1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 7), (6, 8), (6, 9), (6, 10), (7, 8), (7, 9), (7, 10), (8, 9), (8, 10), (9, 10)]


1〜10のなかから異なる2つを選んで順列を作りたい場合も、ほとんどプログラムは同じです。

In [6]:
s = []
for i in range(1,11):
    for j in range(1,11):
        if i != j:
            s.append((i,j))
print(s)

[(1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 1), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 1), (3, 2), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 1), (4, 2), (4, 3), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 1), (5, 2), (5, 3), (5, 4), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 7), (6, 8), (6, 9), (6, 10), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 8), (7, 9), (7, 10), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 9), (8, 10), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 10), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (10, 9)]


In [ ]:
3つを選びだす組み合わせの場合も、2つの場合とほとんど同じです。ここでは0〜5から3つを選んだ組み合わせを列挙します。

In [9]:
s = []
for i in range(6):
    for j in range(i+1,6):
        for k in range(j+1,6):
            s.append((i,j,k))
print(s)
print(len(s))

[(0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 3, 4), (0, 3, 5), (0, 4, 5), (1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 3, 4), (1, 3, 5), (1, 4, 5), (2, 3, 4), (2, 3, 5), (2, 4, 5), (3, 4, 5)]
20


順列の場合は、すこしややこしいです。同じ数が含まれてはいけないので、条件分岐が必要になります。

In [13]:
s = []
for i in range(6):
    for j in range(6):
        if i != j:
            for k in range(j+1,6):
                if i != k and j != k:
                    s.append((i,j,k))
print(s)
print(len(s))

[(0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 3, 4), (0, 3, 5), (0, 4, 5), (1, 0, 2), (1, 0, 3), (1, 0, 4), (1, 0, 5), (1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 3, 4), (1, 3, 5), (1, 4, 5), (2, 0, 1), (2, 0, 3), (2, 0, 4), (2, 0, 5), (2, 1, 3), (2, 1, 4), (2, 1, 5), (2, 3, 4), (2, 3, 5), (2, 4, 5), (3, 0, 1), (3, 0, 2), (3, 0, 4), (3, 0, 5), (3, 1, 2), (3, 1, 4), (3, 1, 5), (3, 2, 4), (3, 2, 5), (3, 4, 5), (4, 0, 1), (4, 0, 2), (4, 0, 3), (4, 0, 5), (4, 1, 2), (4, 1, 3), (4, 1, 5), (4, 2, 3), (4, 2, 5), (4, 3, 5), (5, 0, 1), (5, 0, 2), (5, 0, 3), (5, 0, 4), (5, 1, 2), (5, 1, 3), (5, 1, 4), (5, 2, 3), (5, 2, 4), (5, 3, 4)]
60


順列の場合は、選びだす個数が増えるほど、プログラムが複雑になってきそうですね。集合の性質をつかって、i,j,kがどれも異なる場合のみappendするように書いてみます。

In [14]:
s = []
for i in range(6):
    for j in range(6):
        for k in range(6):
            nums = set([i,j,k])   #set of (i,j,k)
            if len(nums) == 3:    #if it has three members,
                s.append((i,j,k))
print(s)
print(len(s))

[(0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 2, 1), (0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 3, 1), (0, 3, 2), (0, 3, 4), (0, 3, 5), (0, 4, 1), (0, 4, 2), (0, 4, 3), (0, 4, 5), (0, 5, 1), (0, 5, 2), (0, 5, 3), (0, 5, 4), (1, 0, 2), (1, 0, 3), (1, 0, 4), (1, 0, 5), (1, 2, 0), (1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 3, 0), (1, 3, 2), (1, 3, 4), (1, 3, 5), (1, 4, 0), (1, 4, 2), (1, 4, 3), (1, 4, 5), (1, 5, 0), (1, 5, 2), (1, 5, 3), (1, 5, 4), (2, 0, 1), (2, 0, 3), (2, 0, 4), (2, 0, 5), (2, 1, 0), (2, 1, 3), (2, 1, 4), (2, 1, 5), (2, 3, 0), (2, 3, 1), (2, 3, 4), (2, 3, 5), (2, 4, 0), (2, 4, 1), (2, 4, 3), (2, 4, 5), (2, 5, 0), (2, 5, 1), (2, 5, 3), (2, 5, 4), (3, 0, 1), (3, 0, 2), (3, 0, 4), (3, 0, 5), (3, 1, 0), (3, 1, 2), (3, 1, 4), (3, 1, 5), (3, 2, 0), (3, 2, 1), (3, 2, 4), (3, 2, 5), (3, 4, 0), (3, 4, 1), (3, 4, 2), (3, 4, 5), (3, 5, 0), (3, 5, 1), (3, 5, 2), (3, 5, 4), (4, 0, 1), (4, 0, 2), (4, 0, 3), (4, 0, 5), (4, 1, 0), (4, 1, 2), (4, 1, 3), (4, 1, 5), (4, 2, 0), (4, 2, 1), (4, 2, 3)

数字以外のものの組みあわせも、pythonならけっこう簡単です。順列の場合は、ほとんど変更なしです。

In [12]:
members = ('A','B','C','D','E','F')
s = []
for i in members:
    for j in members:
        for k in members:
            nums = set([i,j,k])   #set of (i,j,k)
            if len(nums) == 3:    #if it has three members,
                s.append((i,j,k))
print(s)
print(len(s))

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'B', 'E'), ('A', 'B', 'F'), ('A', 'C', 'B'), ('A', 'C', 'D'), ('A', 'C', 'E'), ('A', 'C', 'F'), ('A', 'D', 'B'), ('A', 'D', 'C'), ('A', 'D', 'E'), ('A', 'D', 'F'), ('A', 'E', 'B'), ('A', 'E', 'C'), ('A', 'E', 'D'), ('A', 'E', 'F'), ('A', 'F', 'B'), ('A', 'F', 'C'), ('A', 'F', 'D'), ('A', 'F', 'E'), ('B', 'A', 'C'), ('B', 'A', 'D'), ('B', 'A', 'E'), ('B', 'A', 'F'), ('B', 'C', 'A'), ('B', 'C', 'D'), ('B', 'C', 'E'), ('B', 'C', 'F'), ('B', 'D', 'A'), ('B', 'D', 'C'), ('B', 'D', 'E'), ('B', 'D', 'F'), ('B', 'E', 'A'), ('B', 'E', 'C'), ('B', 'E', 'D'), ('B', 'E', 'F'), ('B', 'F', 'A'), ('B', 'F', 'C'), ('B', 'F', 'D'), ('B', 'F', 'E'), ('C', 'A', 'B'), ('C', 'A', 'D'), ('C', 'A', 'E'), ('C', 'A', 'F'), ('C', 'B', 'A'), ('C', 'B', 'D'), ('C', 'B', 'E'), ('C', 'B', 'F'), ('C', 'D', 'A'), ('C', 'D', 'B'), ('C', 'D', 'E'), ('C', 'D', 'F'), ('C', 'E', 'A'), ('C', 'E', 'B'), ('C', 'E', 'D'), ('C', 'E', 'F'), ('C', 'F', 'A'), ('C', 'F', 'B'), ('C', 'F', 'D

組み合わせの場合は、繰り返しの範囲を書くのに困ってしまいます。ひとつの方法は次のような書きかたです。

In [15]:
members = ('A','B','C','D','E','F')
N = len(members)
s = []
for i in range(N):
    for j in range(i+1,N):
        for k in range(j+1,N):
            s.append((members[i],members[j],members[k]))
print(s)
print(len(s))

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'B', 'E'), ('A', 'B', 'F'), ('A', 'C', 'D'), ('A', 'C', 'E'), ('A', 'C', 'F'), ('A', 'D', 'E'), ('A', 'D', 'F'), ('A', 'E', 'F'), ('B', 'C', 'D'), ('B', 'C', 'E'), ('B', 'C', 'F'), ('B', 'D', 'E'), ('B', 'D', 'F'), ('B', 'E', 'F'), ('C', 'D', 'E'), ('C', 'D', 'F'), ('C', 'E', 'F'), ('D', 'E', 'F')]
20


さて、はじめのケースに戻りましょう。格子点の座標を決めるプログラムは、重複を許す順列を列挙しているともみなせます。3次元の場合、

In [16]:
pos = []
for x in range(4):
    for y in range(4):
        for z in range(4):
            pos.append((x,y,z))
print(pos)

[(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 0, 3), (0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 1, 3), (0, 2, 0), (0, 2, 1), (0, 2, 2), (0, 2, 3), (0, 3, 0), (0, 3, 1), (0, 3, 2), (0, 3, 3), (1, 0, 0), (1, 0, 1), (1, 0, 2), (1, 0, 3), (1, 1, 0), (1, 1, 1), (1, 1, 2), (1, 1, 3), (1, 2, 0), (1, 2, 1), (1, 2, 2), (1, 2, 3), (1, 3, 0), (1, 3, 1), (1, 3, 2), (1, 3, 3), (2, 0, 0), (2, 0, 1), (2, 0, 2), (2, 0, 3), (2, 1, 0), (2, 1, 1), (2, 1, 2), (2, 1, 3), (2, 2, 0), (2, 2, 1), (2, 2, 2), (2, 2, 3), (2, 3, 0), (2, 3, 1), (2, 3, 2), (2, 3, 3), (3, 0, 0), (3, 0, 1), (3, 0, 2), (3, 0, 3), (3, 1, 0), (3, 1, 1), (3, 1, 2), (3, 1, 3), (3, 2, 0), (3, 2, 1), (3, 2, 2), (3, 2, 3), (3, 3, 0), (3, 3, 1), (3, 3, 2), (3, 3, 3)]


では、d次元のn<sup>d</sup>の格子を出力する、一般的なプログラムは書けるでしょうか?

これはけっこう難しい問題です。Fortranのような古い言語ではトリッキーな方法を使わざるをえません。Pythonでも、再帰が必要になります。

In [ ]:
#d次元の、n^d個の格子点の座標をリストの形で返す関数。
def lattice(n,d):
    #1次元の場合には、0〜n-1を返す。
    if d == 1:
        return [[i,] for i in range(n)]

lattice(10,1)    

2次元以上の場合は、n-1次元の座標にもう1つ要素を追加する。

In [ ]:
#d次元の、n^d個の格子点の座標をリストの形で返す関数。
def lattice(n,d):
    #1次元の場合には、0〜n-1を返す。
    if d == 1:
        return [[i,] for i in range(n)]
    points = []
    #n次元の座標は、n-1次元の座標に1次元の座標を付けくわえる
    points0 = lattice(n,d-1)
    for i in lattice(n,1):
        for j in points0:
            points.append(i+j)
    return points
lattice(4,3)

このようなプログラムは、なかなか間違いなく書くのは難しく、読むにもすこし経験が必要です。

そこで、標準ライブラリitertoolsの出番です。
##itertoolsの概要
itertoolsは、順列組み合わせの基本的なアルゴリズムを集めたものです。→https://docs.python.org/3/library/itertools.html

###組みあわせ
5つの要素のなかから、2つを選ぶ組みあわせは、Fortranなどでは2重ループで書くのが一般的ですが、itertoolsを使うと非常に簡単に書けます。

In [ ]:
import itertools
for a,b in itertools.combinations([1,2,3,4,5], 2):
    print(a,b)

##順列
5つの要素を並びかえるすべての順列に対して何かの処理をしたい場合も、繰り返しで書くのは骨がおれますが、itertoolsではとても簡単です。

In [ ]:
for p in itertools.permutations("ドレミソラ"):
    print(p)

##重複順列
4x4x4の格子点を生成する問題は、重複順列とみなすことができます。itertoolsには重複順列ももちろんあります。

In [ ]:
for i in itertools.product([0,1,2,3],[0,1,2,3],[0,1,2,3]):
    print(i)

いくつも同じものを並べるのが面倒なので、繰り返し回数を指定します。

In [ ]:
for i in itertools.product(range(4),repeat=3):
    print(i)

##使用例1

1次元のランダムウォークは、1歩ごとに右(+1)か左(-1)に進みます。10歩のランダムウォークの全経路は、

In [ ]:
for i in itertools.product([-1,+1],repeat=10):
    print(i)

和をとると、最終的な座標がわかります。

In [ ]:
for i in itertools.product([-1,+1],repeat=10):
    print(sum(i))

これを集計すると、2項分布になることがわかります。repeat=の部分を大きくするとなめらかになりますが、20を越えると暴走するかもしれません。

In [ ]:
%matplotlib inline
import pylab
positions = dict()
for i in itertools.product([-1,+1],repeat=10):
    pos = sum(i)
    if pos in positions:
        positions[pos] += 1
    else:
        positions[pos] = 1
#辞書のKeyをソートしたもの
x = sorted(positions)
#対応する累積数
y = [positions[pos] for pos in x]

pylab.plot(x,y)

##使用例2
にせアンパンマンをたくさん作ります。join関数は、文字列のリストをひとつの文字列に合体させます。

In [ ]:
import itertools
for i in itertools.product(["ア","マ"],["リ","ソ","ン"],["パ","バ"],["リ","ソ","ン"],["ア","マ"],["リ","ソ","ン"]):
    print(".".join(i))